In [1]:
import pandas as pd
import json

In [2]:
with open('result.json') as file:
    data = json.load(file)
df = pd.DataFrame(data)
df

,filename,frame_id,objects
0,sample_imgs/IMG_6754-X3.jpg,1,"[{'class_id': 0, 'name': 'bib', 'relative_coor..."
1,sample_imgs/IMG_7350-X3.jpg,2,"[{'class_id': 0, 'name': 'bib', 'relative_coor..."
2,sample_imgs/IMG_7815-X3.jpg,3,"[{'class_id': 0, 'name': 'bib', 'relative_coor..."


In [3]:
med=[]
def check(x):
    for y in x:
        if y['name']=="bib":
                    return True
    return False
df['bibs']=df['objects'].transform(lambda temp:check(temp))

In [4]:
df

,filename,frame_id,objects,bibs
0,sample_imgs/IMG_6754-X3.jpg,1,"[{'class_id': 0, 'name': 'bib', 'relative_coor...",True
1,sample_imgs/IMG_7350-X3.jpg,2,"[{'class_id': 0, 'name': 'bib', 'relative_coor...",True
2,sample_imgs/IMG_7815-X3.jpg,3,"[{'class_id': 0, 'name': 'bib', 'relative_coor...",True


In [5]:
df['filename']=df['filename'].transform(lambda f:f.split('/')[-1])
in_book=list(df['filename'])

In [6]:
df

,filename,frame_id,objects,bibs
0,IMG_6754-X3.jpg,1,"[{'class_id': 0, 'name': 'bib', 'relative_coor...",True
1,IMG_7350-X3.jpg,2,"[{'class_id': 0, 'name': 'bib', 'relative_coor...",True
2,IMG_7815-X3.jpg,3,"[{'class_id': 0, 'name': 'bib', 'relative_coor...",True


In [7]:
med=[]
def check2(x):
    for y in x:
        if y['name']=="bib":
                    return y
    return False
df['bib_data']=df['objects'].transform(lambda temp:check2(temp))

In [8]:
df

,filename,frame_id,objects,bibs,bib_data
0,IMG_6754-X3.jpg,1,"[{'class_id': 0, 'name': 'bib', 'relative_coor...",True,"{'class_id': 0, 'name': 'bib', 'relative_coord..."
1,IMG_7350-X3.jpg,2,"[{'class_id': 0, 'name': 'bib', 'relative_coor...",True,"{'class_id': 0, 'name': 'bib', 'relative_coord..."
2,IMG_7815-X3.jpg,3,"[{'class_id': 0, 'name': 'bib', 'relative_coor...",True,"{'class_id': 0, 'name': 'bib', 'relative_coord..."


In [9]:
df=df.drop(['frame_id','objects','bibs'],axis=1)


In [10]:
df

,filename,bib_data
0,IMG_6754-X3.jpg,"{'class_id': 0, 'name': 'bib', 'relative_coord..."
1,IMG_7350-X3.jpg,"{'class_id': 0, 'name': 'bib', 'relative_coord..."
2,IMG_7815-X3.jpg,"{'class_id': 0, 'name': 'bib', 'relative_coord..."


In [11]:
data=df

In [12]:
def details(x,flag_2):
        co=x['relative_coordinates']
        wid=float(co['width'])/2.0
        hi=float(co['height'])/2.0
        if flag_2=='y2':
            return float(co['top_y'])+hi
        if flag_2=='x2':
            return float(co['left_x'])+wid
        if flag_2=='x1':
            return float(co['left_x'])-wid
        if flag_2=='y1':
            return float(co['top_y'])-hi
data['x1']=data['bib_data'].transform(lambda f:details(f,'x1'))    
data['y1']=data['bib_data'].transform(lambda f:details(f,'y1'))  
data['x2']=data['bib_data'].transform(lambda f:details(f,'x2'))  
data['y2']=data['bib_data'].transform(lambda f:details(f,'y2'))  

In [15]:
data

,filename,bib_data,x1,y1,x2,y2
0,IMG_6754-X3.jpg,"{'class_id': 0, 'name': 'bib', 'relative_coord...",671.5,478.5,760.5,521.5
1,IMG_7350-X3.jpg,"{'class_id': 0, 'name': 'bib', 'relative_coord...",735.5,430.5,808.5,477.5
2,IMG_7815-X3.jpg,"{'class_id': 0, 'name': 'bib', 'relative_coord...",345.5,435.5,426.5,486.5


In [20]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = 9927331200
count=0
for x,y in data.iterrows():
        img = Image.open('./images/'+str(y['filename']))
        dim=img.size
        w,h=dim[0],dim[1]
        x1=y['x1']*w
        x2=y['x2']*w
        y1=y['y1']*h
        y2=y['y2']*h
        area=(x1,y1,x2,y2)
        print(count)
        cropped_img = img.crop(area)
        cropped_img=cropped_img.convert('RGB')
        cropped_img.save("./cropped_images/"+str(x)+'.jpg')
        count+=1

0


MemoryError: 

In [14]:
import cv2
import matplotlib.pyplot as plt

x=716
y=500
w=89
h=43
img = cv2.imread('IMG_6754-X3.jpg')
crop_img = img[y:y+h, x:x+w]
cv2.imwrite("Object.jpg",crop_img)

True

In [2]:
import pandas as pd
import json
with open('result.json') as file:
    data = json.load(file)

In [21]:
cropped_images_df = pd.DataFrame(columns=['filename','left_x','top_y','width','height'])
cropped_images_list=[]
for items in data:
    filename=items['filename'].split('/')[-1]
    for item in items['objects']:
        if item['name']=='bib':
            dict_info=item['relative_coordinates']
            dict_info['filename']=filename
            cropped_images_list.append(dict_info)

In [24]:
cropped_images_df=pd.DataFrame(cropped_images_list)
print(cropped_images_df)

            filename  height  left_x  top_y  width
0    IMG_6688-X3.jpg      39     814    477     58
1    IMG_6689-X3.jpg      35     655    546     66
2    IMG_6692-X3.jpg      34     899    359     75
3    IMG_6695-X3.jpg      48     707    340    107
4    IMG_6699-X3.jpg      41    1064    404     98
5    IMG_6705-X3.jpg      54     682    372     67
6    IMG_6710-X3.jpg      38     781    376     52
7    IMG_6711-X3.jpg      54     883    380     89
8    IMG_6716-X3.jpg      33     774    417     57
9    IMG_6720-X3.jpg      47     656    352     86
10   IMG_6724-X3.jpg      51     454    379     90
11   IMG_6724-X3.jpg      33     608    405     62
12   IMG_6736-X3.jpg      37     465    511     64
13   IMG_6738-X3.jpg      52     773    524     93
14   IMG_6741-X3.jpg      49     749    417     91
15   IMG_6743-X3.jpg      42     649    494     91
16   IMG_6746-X3.jpg      32     839    587     54
17   IMG_6747-X3.jpg      53     775    429     91
18   IMG_6748-X3.jpg      54   

In [32]:
import cv2
import matplotlib.pyplot as plt

x=814
y=477
w=58
h=39
dir_name='cropped_images'
img = cv2.imread('images\IMG_6688-X3.jpg')
crop_img = img[y:y+h, x:x+w]
cv2.imwrite("Object.jpg",crop_img)

True

In [50]:
import cv2
dir_name='cropped_images'
for index, row in cropped_images_df.iterrows():
    source='images'+'\\'+(row['filename'])
    img = cv2.imread(source)
    x=row['left_x']
    y=row['top_y']
    w=row['width']
    h=row['height']
    crop_img = img[y:y+h, x:x+w]
    cropped_image_name=dir_name+'\\'+ row['filename'].split('.')[0]+'_'+str(row['left_x'])+'.jpg'
    cv2.imwrite(cropped_image_name,crop_img)                

In [49]:
import cv2
dir_name='cropped_images'
for index, row in cropped_images_df.iterrows():
    print('images'+'\\'+row['filename'].split('.')[0]+'_'+str(row['left_x'])+'.jpg')
    

images\IMG_6688-X3_814.jpg
images\IMG_6689-X3_655.jpg
images\IMG_6692-X3_899.jpg
images\IMG_6695-X3_707.jpg
images\IMG_6699-X3_1064.jpg
images\IMG_6705-X3_682.jpg
images\IMG_6710-X3_781.jpg
images\IMG_6711-X3_883.jpg
images\IMG_6716-X3_774.jpg
images\IMG_6720-X3_656.jpg
images\IMG_6724-X3_454.jpg
images\IMG_6724-X3_608.jpg
images\IMG_6736-X3_465.jpg
images\IMG_6738-X3_773.jpg
images\IMG_6741-X3_749.jpg
images\IMG_6743-X3_649.jpg
images\IMG_6746-X3_839.jpg
images\IMG_6747-X3_775.jpg
images\IMG_6748-X3_764.jpg
images\IMG_6749-X3_742.jpg
images\IMG_6751-X3_750.jpg
images\IMG_6754-X3_716.jpg
images\IMG_6759-X3_621.jpg
images\IMG_6760-X3_628.jpg
images\IMG_6763-X3_555.jpg
images\IMG_6763-X3_995.jpg
images\IMG_6768-X3_644.jpg
images\IMG_6778-X3_427.jpg
images\IMG_6778-X3_913.jpg
images\IMG_6779-X3_453.jpg
images\IMG_6779-X3_894.jpg
images\IMG_6781-X3_670.jpg
images\IMG_6782-X3_475.jpg
images\IMG_6786-X3_707.jpg
images\IMG_6797-X3_470.jpg
images\IMG_6797-X3_772.jpg
images\IMG_6798-X3_557.jpg
